In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter

# Configuration settings
class Config:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    lr = 3e-4
    zDim = 128  # Latent vector size
    imageDim = 28
    batchSize = 32
    numEpochs = 100
    logstep = 625

config = Config()

# Define transformations
myTransform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

# Load MNIST dataset
dataset = datasets.MNIST(root="dataset/", transform=myTransform, download=True)
loader = DataLoader(dataset, batch_size=config.batchSize, shuffle=True)

# CNN-based Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# CNN-based Generator
class Generator(nn.Module):
    def __init__(self, zDim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(zDim, 128 * 7 * 7),
            nn.BatchNorm1d(128 * 7 * 7),
            nn.ReLU(),
            nn.Unflatten(1, (128, 7, 7)),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 1, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

# Initialize models
discriminator = Discriminator().to(config.device)
generator = Generator(config.zDim).to(config.device)
fixedNoise = torch.randn((config.batchSize, config.zDim)).to(config.device)

# Optimizers and loss function
optDisc = optim.Adam(discriminator.parameters(), lr=config.lr)
optGen = optim.Adam(generator.parameters(), lr=config.lr)
criterion = nn.BCELoss()

# Summary writers
writerFake = SummaryWriter("C:\\Users\\Muraa\\Desktop\\CNN\\Fake")
writerReal = SummaryWriter("C:\\Users\\Muraa\\Desktop\\CNN\\Real")

print("Started Training and Visualization...")
step = 0

for epoch in range(config.numEpochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.to(config.device)
        batch_size = real.size(0)

        # Train Discriminator
        noise = torch.randn(batch_size, config.zDim).to(config.device)
        fake = generator(noise)

        discReal = discriminator(real).view(-1)
        discFake = discriminator(fake.detach()).view(-1)

        lossDreal = criterion(discReal, torch.ones_like(discReal))
        lossDfake = criterion(discFake, torch.zeros_like(discFake))
        lossD = (lossDreal + lossDfake) / 2

        discriminator.zero_grad()
        lossD.backward()
        optDisc.step()

        # Train Generator
        output = discriminator(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))

        generator.zero_grad()
        lossG.backward()
        optGen.step()

        # Logging and TensorBoard visualization
        if batch_idx % config.logstep == 0:
            print(f"Epoch [{epoch}/{config.numEpochs}] Batch {batch_idx}/{len(loader)} Loss D: {lossD:.4f}, Loss G: {lossG:.4f}")

            with torch.no_grad():
                fake = generator(fixedNoise).reshape(-1, 1, 28, 28)
                real = real.reshape(-1, 1, 28, 28)
                
                imgGridFake = torchvision.utils.make_grid(fake, normalize=True)
                imgGridReal = torchvision.utils.make_grid(real, normalize=True)
                
                writerFake.add_image("MNIST Fake Images", imgGridFake, global_step=step)
                writerReal.add_image("MNIST Real Images", imgGridReal, global_step=step)

                writerFake.flush()
                writerReal.flush()

            step += 1

writerFake.close()
writerReal.close()



Started Training and Visualization...
Epoch [0/100] Batch 0/1875 Loss D: 0.6450, Loss G: 1.1256
Epoch [0/100] Batch 625/1875 Loss D: 0.0001, Loss G: 9.4397
Epoch [0/100] Batch 1250/1875 Loss D: 0.0000, Loss G: 10.7337
Epoch [1/100] Batch 0/1875 Loss D: 0.0000, Loss G: 33.2033
Epoch [1/100] Batch 625/1875 Loss D: 0.0000, Loss G: 15.1516
Epoch [1/100] Batch 1250/1875 Loss D: 0.0000, Loss G: 46.0994
Epoch [2/100] Batch 0/1875 Loss D: 0.0000, Loss G: 45.8353
Epoch [2/100] Batch 625/1875 Loss D: 0.0000, Loss G: 45.4560
Epoch [2/100] Batch 1250/1875 Loss D: 0.0000, Loss G: 45.3596
Epoch [3/100] Batch 0/1875 Loss D: 0.0000, Loss G: 45.4050
Epoch [3/100] Batch 625/1875 Loss D: 0.0000, Loss G: 45.4742
Epoch [3/100] Batch 1250/1875 Loss D: 0.0000, Loss G: 45.2892
Epoch [4/100] Batch 0/1875 Loss D: 0.0000, Loss G: 45.2095
Epoch [4/100] Batch 625/1875 Loss D: 0.0000, Loss G: 44.9964
Epoch [4/100] Batch 1250/1875 Loss D: 0.0000, Loss G: 44.6367
Epoch [5/100] Batch 0/1875 Loss D: 0.0000, Loss G: 45

In [4]:
tensorboard --logdir="C:\Users\Muraa\Desktop\CNN" --bind_all

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3154903059.py, line 1)

In [ ]:
set TF_ENABLE_ONEDNN_OPTS=0
